<a href="https://colab.research.google.com/github/Lmalviya/machineTranslationTask/blob/main/machineTranslationUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>